# Import libraries and packages

In [1]:
import json
import time 

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional

from selenium import webdriver

from supabase import create_client, Client


In [2]:
driver = webdriver.Firefox()

In [3]:
# url to scrape
whoscored_url = "https://www.whoscored.com/Matches/1729371/Live/England-Premier-League-2023-2024-Nottingham-Forest-Liverpool"
driver.get(whoscored_url)

In [4]:
# create soup to parse
soup = BeautifulSoup(driver.page_source, "html.parser")

In [5]:
# select the script tag that contains the matchCentreData
element = soup.select_one("script:-soup-contains('matchCentreData')")

In [6]:
# parse the matchCentreData
matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(",\n")[0])

In [7]:
# check the keys of the matchdict
matchdict.keys() 

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [8]:
# check a random event
matchdict['events'][55]

{'id': 2653362563.0,
 'eventId': 21,
 'minute': 3,
 'second': 43,
 'teamId': 174,
 'playerId': 124688,
 'x': 16.4,
 'y': 8.3,
 'expandedMinute': 3,
 'period': {'value': 1, 'displayName': 'FirstHalf'},
 'type': {'value': 49, 'displayName': 'BallRecovery'},
 'outcomeType': {'value': 1, 'displayName': 'Successful'},
 'qualifiers': [],
 'satisfiedEventsTypes': [93],
 'isTouch': False}

In [9]:
match_events = matchdict['events']

In [10]:
# dictionary of the events
match_events

[{'id': 2653354707.0,
  'eventId': 2,
  'minute': 0,
  'second': 0,
  'teamId': 26,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2653354685.0,
  'eventId': 2,
  'minute': 0,
  'second': 0,
  'teamId': 174,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2653354759.0,
  'eventId': 3,
  'minute': 0,
  'second': 0,
  'teamId': 174,
  'playerId': 342830,
  'x': 50.0,
  'y': 50.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 1, 'displayName': 'Pass'},
  'outcomeTy

In [11]:
match_players = matchdict['home']['players'] + matchdict['away']['players']
match_players

[{'playerId': 78386,
  'shirtNo': 26,
  'name': 'Matz Sels',
  'position': 'GK',
  'height': 188,
  'weight': 75,
  'age': 32,
  'isFirstEleven': True,
  'isManOfTheMatch': False,
  'field': 'home',
  'stats': {'totalSaves': {'96': 1.0},
   'parriedSafe': {'96': 1.0},
   'possession': {'0': 1.0,
    '2': 1.0,
    '6': 1.0,
    '8': 1.0,
    '13': 1.0,
    '15': 1.0,
    '18': 1.0,
    '19': 2.0,
    '24': 1.0,
    '32': 1.0,
    '34': 1.0,
    '35': 1.0,
    '49': 1.0,
    '52': 1.0,
    '53': 1.0,
    '56': 1.0,
    '60': 1.0,
    '62': 2.0,
    '66': 1.0,
    '69': 1.0,
    '72': 1.0,
    '74': 1.0,
    '77': 1.0,
    '89': 1.0,
    '93': 2.0,
    '96': 1.0},
   'ratings': {'0': 6.01,
    '2': 6.0,
    '6': 6.01,
    '8': 6.02,
    '13': 6.03,
    '15': 6.03,
    '18': 6.04,
    '19': 6.05,
    '24': 6.04,
    '32': 6.05,
    '34': 6.04,
    '35': 6.06,
    '49': 6.04,
    '52': 6.05,
    '53': 6.04,
    '56': 6.03,
    '59': 6.03,
    '60': 6.02,
    '62': 6.0,
    '66': 5.99,
    '

In [12]:
df_players = pd.DataFrame(match_players)
df_players.head()

,playerId,shirtNo,name,position,height,weight,age,isFirstEleven,isManOfTheMatch,field,stats,subbedInPlayerId,subbedOutPeriod,subbedOutExpandedMinute,subbedInPeriod,subbedInExpandedMinute,subbedOutPlayerId
0,78386,26,Matz Sels,GK,188,75,32,True,False,home,"{'totalSaves': {'96': 1.0}, 'parriedSafe': {'9...",NaN,NaN,NaN,NaN,NaN,NaN
1,386969,7,Neco Williams,DR,183,72,22,True,False,home,"{'possession': {'0': 1.0, '1': 1.0, '6': 1.0, ...",NaN,NaN,NaN,NaN,NaN,NaN
2,399490,32,Andrew Omobamidele,DC,188,0,21,True,False,home,"{'possession': {'1': 1.0, '13': 1.0, '20': 1.0...",NaN,NaN,NaN,NaN,NaN,NaN
3,445424,40,Murillo,DC,180,75,21,True,False,home,"{'possession': {'0': 2.0, '8': 1.0, '10': 1.0,...",NaN,NaN,NaN,NaN,NaN,NaN
4,298659,15,Harry Toffolo,DL,183,71,28,True,False,home,"{'possession': {'7': 1.0, '10': 1.0, '12': 1.0...",NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_players.columns

Index(['playerId', 'shirtNo', 'name', 'position', 'height', 'weight', 'age',
       'isFirstEleven', 'isManOfTheMatch', 'field', 'stats',
       'subbedInPlayerId', 'subbedOutPeriod', 'subbedOutExpandedMinute',
       'subbedInPeriod', 'subbedInExpandedMinute', 'subbedOutPlayerId'],
      dtype='object')

In [14]:
df_players = df_players[['playerId','name','position']]
df_players.head()

,playerId,name,position
0,78386,Matz Sels,GK
1,386969,Neco Williams,DR
2,399490,Andrew Omobamidele,DC
3,445424,Murillo,DC
4,298659,Harry Toffolo,DL


In [15]:
# create a df from the match_events
df = pd.DataFrame(match_events)
df.head()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,endY,relatedEventId,relatedPlayerId,goalMouthZ,goalMouthY,isShot,blockedX,blockedY,cardType,isGoal
0,2.653355e+09,2,0,0.0,26,0.0,0.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 32, 'displayName': 'Start'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.653355e+09,2,0,0.0,174,0.0,0.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 32, 'displayName': 'Start'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.653355e+09,3,0,0.0,174,50.0,50.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,53.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.653355e+09,4,0,3.0,174,33.9,56.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,72.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.653355e+09,5,0,5.0,174,44.7,73.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,67.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'relatedEventId', 'relatedPlayerId', 'goalMouthZ', 'goalMouthY',
       'isShot', 'blockedX', 'blockedY', 'cardType', 'isGoal'],
      dtype='object')

# Data cleansing

In [17]:
df_players.isnull().sum()

playerId    0
name        0
position    0
dtype: int64

In [18]:
# check null values
df.isnull().sum()

id                         0
eventId                    0
minute                     0
second                     3
teamId                     0
x                          0
y                          0
expandedMinute             0
period                     0
type                       0
outcomeType                0
qualifiers                 0
satisfiedEventsTypes       0
isTouch                    0
playerId                  16
endX                     554
endY                     554
relatedEventId          1587
relatedPlayerId         1587
goalMouthZ              1599
goalMouthY              1599
isShot                  1599
blockedX                1608
blockedY                1608
cardType                1624
isGoal                  1628
dtype: int64

In [19]:
df.dropna(subset=['playerId'], inplace=True)

In [20]:
df.isnull().sum()

id                         0
eventId                    0
minute                     0
second                     3
teamId                     0
x                          0
y                          0
expandedMinute             0
period                     0
type                       0
outcomeType                0
qualifiers                 0
satisfiedEventsTypes       0
isTouch                    0
playerId                   0
endX                     538
endY                     538
relatedEventId          1571
relatedPlayerId         1571
goalMouthZ              1583
goalMouthY              1583
isShot                  1583
blockedX                1592
blockedY                1592
cardType                1608
isGoal                  1612
dtype: int64

In [21]:
df = df.where(pd.notnull(df), None)

In [22]:
# merge the df with the df_players in player_id
df = df.merge(df_players, left_on='playerId', right_on='playerId', how='left')
df.head()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,relatedPlayerId,goalMouthZ,goalMouthY,isShot,blockedX,blockedY,cardType,isGoal,name,position
0,2.653355e+09,3,0,0.0,174,50.0,50.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,NaN,NaN,NaN,None,NaN,NaN,None,None,Ryan Yates,DMC
1,2.653355e+09,4,0,3.0,174,33.9,56.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,NaN,NaN,NaN,None,NaN,NaN,None,None,Murillo,DC
2,2.653355e+09,5,0,5.0,174,44.7,73.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,NaN,NaN,NaN,None,NaN,NaN,None,None,Nicolás Domínguez,DMC
3,2.653355e+09,6,0,6.0,174,53.4,67.7,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,NaN,NaN,NaN,None,NaN,NaN,None,None,Morgan Gibbs-White,AMC
4,2.653355e+09,7,0,8.0,174,25.0,69.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,NaN,NaN,NaN,None,NaN,NaN,None,None,Murillo,DC


In [23]:
df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'relatedEventId', 'relatedPlayerId', 'goalMouthZ', 'goalMouthY',
       'isShot', 'blockedX', 'blockedY', 'cardType', 'isGoal', 'name',
       'position'],
      dtype='object')

In [24]:
df = df.rename(
    {
    
    'eventId': 'event_id',
    'expandedMinute': 'expanded_minute',
    'outcomeType': 'outcome_type',
    'isTouch': 'is_touch',
    'playerId': 'player_id',
    'teamId': 'team_id',
    'endX': 'end_x',
    'endY': 'end_y',
    'blockedX': 'blocked_x',
    'blockedY': 'blocked_y',
    'goalMouthY': 'goal_mouth_y',
    'goalMouthZ': 'goal_mouth_z',
    'isShot': 'is_shot',
    'cardType': 'card_type',
    'isGoal': 'is_goal',
    'name': 'player_name'
},
axis=1
)


In [25]:
# create new columns for the nested dictionaries
df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])

In [26]:
df['type_display_name'].head()

0    Pass
1    Pass
2    Pass
3    Pass
4    Pass
Name: type_display_name, dtype: object

In [27]:
# renmove the nested dictionaries
df.drop(columns=['period', 'type', 'outcome_type'], inplace=True)

In [30]:
df = df[['id','player_name','event_id','minute','second','team_id','player_id','x','y','end_x','end_y',
         'qualifiers','is_touch','blocked_x','blocked_y','goal_mouth_y','goal_mouth_z','is_shot',
         'card_type','is_goal','type_display_name','outcome_type_display_name',
         'period_display_name']]

In [31]:
df.dtypes

id                           float64
player_name                   object
event_id                       int64
minute                         int64
second                       float64
team_id                        int64
player_id                    float64
x                            float64
y                            float64
end_x                        float64
end_y                        float64
qualifiers                    object
is_touch                        bool
blocked_x                    float64
blocked_y                    float64
goal_mouth_y                 float64
goal_mouth_z                 float64
is_shot                       object
card_type                     object
is_goal                       object
type_display_name             object
outcome_type_display_name     object
period_display_name           object
dtype: object

In [39]:
df[['id', 'event_id', 'minute', 'team_id', 'player_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id']].astype(np.int64)

C:\Users\imjos\AppData\Local\Temp\ipykernel_33992\452101415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['id', 'event_id', 'minute', 'team_id', 'player_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id']].astype(np.int64)


In [40]:
df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype('float')

C:\Users\imjos\AppData\Local\Temp\ipykernel_33992\1649267266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype('float')


In [41]:
df[['is_shot','is_goal','card_type']] = df[['is_shot','is_goal','card_type']].astype('bool')

C:\Users\imjos\AppData\Local\Temp\ipykernel_33992\1341704250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['is_shot','is_goal','card_type']] = df[['is_shot','is_goal','card_type']].astype('bool')


In [42]:
df['is_goal'] = df['is_goal'].fillna(False)
df['is_shot'] = df['is_shot'].fillna(False)

C:\Users\imjos\AppData\Local\Temp\ipykernel_33992\1451954947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_goal'] = df['is_goal'].fillna(False)
C:\Users\imjos\AppData\Local\Temp\ipykernel_33992\1451954947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_shot'] = df['is_shot'].fillna(False)


In [43]:
# loop for to check the null values
for column in df.columns:
    if df[column].dtype == np.float64 or df[column].dtype == np.float32:
        df[column] = np.where(
            np.isnan(df[column]),
            None,
            df[column]
        )

C:\Users\imjos\AppData\Local\Temp\ipykernel_33992\910020158.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.where(


In [44]:
df

,id,player_name,event_id,minute,second,team_id,player_id,x,y,end_x,...,blocked_x,blocked_y,goal_mouth_y,goal_mouth_z,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name
0,-1641612537,Ryan Yates,3,0,0.0,174,342830,50.0,50.0,33.0,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
1,-1641612481,Murillo,4,0,3.0,174,445424,33.9,56.0,45.9,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
2,-1641612429,Nicolás Domínguez,5,0,5.0,174,334653,44.7,73.0,52.4,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
3,-1641612409,Morgan Gibbs-White,6,0,6.0,174,332867,53.4,67.7,33.2,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
4,-1641612329,Murillo,7,0,8.0,174,445424,25.0,69.0,12.6,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,-1641362897,Callum Hudson-Odoi,716,100,39.0,174,350088,58.6,81.7,68.2,...,None,None,None,None,False,False,False,Pass,Unsuccessful,SecondHalf
1609,-1641362855,Luis Díaz,1095,100,40.0,26,377168,27.7,59.3,36.6,...,None,None,None,None,False,False,False,Pass,Successful,SecondHalf
1610,-1641362841,Darwin Núñez,1096,100,42.0,26,400828,36.4,49.0,None,...,None,None,None,None,False,False,False,BallRecovery,Successful,SecondHalf
1611,-1641362735,Darwin Núñez,1097,100,45.0,26,400828,36.0,55.2,42.0,...,None,None,None,None,False,False,False,Pass,Successful,SecondHalf
